In [1]:
import sys

In [2]:
sys.path.append("../Traffic4cast")

In [3]:
sys.path

['/gpfs/space/home/pyvovar/repos/TrafficForecast/notebooks',
 '/gpfs/space/home/pyvovar/miniconda3/envs/t4c22/lib/python38.zip',
 '/gpfs/space/home/pyvovar/miniconda3/envs/t4c22/lib/python3.8',
 '/gpfs/space/home/pyvovar/miniconda3/envs/t4c22/lib/python3.8/lib-dynload',
 '',
 '/gpfs/space/home/pyvovar/.local/lib/python3.8/site-packages',
 '/gpfs/space/home/pyvovar/miniconda3/envs/t4c22/lib/python3.8/site-packages',
 '../Traffic4cast']

In [4]:
import os
import argparse

import statistics
from collections import defaultdict

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import torch
import torch.nn.functional as F
import torch_geometric
import tqdm
from torch import nn
from pathlib import Path
import numpy as np
import random
from sklearn.model_selection import KFold

import t4c22
from t4c22.metric.masked_crossentropy import get_weights_from_class_fractions
from t4c22.misc.t4c22_logging import t4c_apply_basic_logging_config
from t4c22.t4c22_config import class_fractions
from t4c22.t4c22_config import load_basedir
from t4c22.dataloading.t4c22_dataset_geometric import T4c22GeometricDataset

t4c_apply_basic_logging_config(loglevel="DEBUG")

BASEDIR = load_basedir(fn="t4c22_config.json", pkg=t4c22)

In [5]:
torch.__version__

'2.1.2'

In [6]:
torch.cuda.is_available()

True

In [5]:
HIDDEN_CHANNELS = 32
NUM_LAYERS = 3
NUM_EDGE_CLASSES = 3
NUM_FEATURES = 4
DROPOUT = 0.0
BATCH_SIZE = 2
CITY = "london"
nan_to_num_map = {"london": -1.21, "melbourne": -0.8, "madrid": -0.56}
SEED = 1234
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.cuda.manual_seed(SEED)
device = torch.device("cuda", 0)

In [6]:
BASEDIR

PosixPath('/gpfs/space/home/pyvovar/repos/TrafficForecast/data')

In [7]:
class RecLinear(nn.Module):
    def __init__(self, num_edges, num_nodes, num_attrs, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):

        super(RecLinear, self).__init__()

        self.embed = nn.Embedding(num_edges, hidden_channels)
        self.node_embed = nn.Embedding(num_nodes, hidden_channels)
        self.node_embed1 = nn.Embedding(num_nodes, 4)
        self.time_embed = nn.Embedding(96, hidden_channels)
        self.week_embed = nn.Embedding(7, hidden_channels)
        self.node_index = torch.arange(0, num_nodes).to(device)

        self.node_lin = nn.Linear(in_channels, hidden_channels)
        self.node_lin1 = nn.Linear(hidden_channels * 2, hidden_channels)
        self.attr_lin = nn.Linear(num_attrs, hidden_channels)
        self.attr_lin1 = nn.Sequential(nn.Linear(num_attrs, hidden_channels), nn.LeakyReLU(),
                                       nn.Linear(hidden_channels, hidden_channels))
        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(hidden_channels * 6, hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.fc1 = nn.Linear(num_nodes, 256)
        self.fc2 = nn.Linear(256, 32)  # mean vector
        self.fc3 = nn.Linear(256, 32)  # standard deviation vector
        self.fc4 = nn.Linear(32, 256)
        self.fc5 = nn.Linear(256, num_nodes)

        from torch_geometric.nn import GATv2Conv as GCNConv

        self.conv1 = torch.nn.ModuleList()
        for i in range(3):
            self.conv1.append(GCNConv(hidden_channels, hidden_channels, edge_dim=hidden_channels))

        self.conv2 = torch.nn.ModuleList()
        for i in range(3):
            self.conv2.append(GCNConv(hidden_channels, hidden_channels, edge_dim=hidden_channels))

        self.gcn_lin1 = nn.Linear(hidden_channels * 2, hidden_channels)
        self.gcn_lin2 = nn.Linear(hidden_channels * 2, hidden_channels)

    def gelu(self, x):
        return 0.5 * x * (1 + F.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * torch.pow(x, 3))))

    # coding process
    def encode(self, x):
        h = self.gelu(self.fc1(x))
        return self.fc2(h), self.fc3(h)

    # Gaussian sampling
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var / 2)
        eps = torch.randn_like(std)
        return mu + eps * std

    # decoding process
    def decode(self, z):
        h = self.gelu(self.fc4(z))
        h = self.fc5(h)
        return h

    def reset_parameters(self):
        self.embed.reset_parameters()
        self.node_embed.reset_parameters()
        self.node_embed1.reset_parameters()
        self.time_embed.reset_parameters()
        self.node_lin.reset_parameters()
        self.node_lin1.reset_parameters()
        self.attr_lin.reset_parameters()
        self.fc1.reset_parameters()
        self.fc2.reset_parameters()
        self.fc3.reset_parameters()
        self.fc4.reset_parameters()
        self.fc5.reset_parameters()
        for lin in self.lins:
            lin.reset_parameters()
        for lin in self.conv1:
            lin.reset_parameters()
        for lin in self.conv2:
            lin.reset_parameters()
        self.gcn_lin1.reset_parameters()
        self.gcn_lin2.reset_parameters()

    def forward(self, index, edge_index, x, attr, cur_t, cur_w):
        mask_idx = (torch.sum(x, dim=1, keepdim=True) != nan_to_num_map[CITY] * 4).type(torch.float)

        xmax = 23.91
        xmin = nan_to_num_map[CITY]
        x_norm = (x - xmin) / (xmax - xmin)

        ratio = 0.8 + 0.4 * np.random.rand(1)[0]

        x_norm = x_norm * ratio

        drop_idx = (torch.rand_like(x_norm[:, 0:1]) > 0.4).type(torch.float)
        x_norm = x_norm * drop_idx

        x_norm = torch.transpose(x_norm, 0, 1)
        mu, log_var = self.encode(x_norm)
        z = self.reparameterize(mu, log_var)
        x_rec = self.decode(z)

        x_rec = x_rec / ratio

        x_rec = torch.transpose(x_rec, 0, 1)
        x_rec = x_rec * (xmax - xmin) + xmin
        x_rec1 = mask_idx * x + (1 - mask_idx) * x_rec

        attr1 = self.attr_lin(attr)
        embed = self.embed(index)

        node_embed = self.node_embed(self.node_index)
        pre_data = node_embed
        for conv in self.conv1:
            node_embed = conv(node_embed, edge_index, attr1)
            node_embed = self.gelu(node_embed) + pre_data

        data = self.gelu(self.node_lin(x_rec1))
        pre_data = data
        for conv in self.conv2:
            data = conv(data, edge_index, attr1)
            data = self.gelu(data) + pre_data

        x_i = torch.index_select(data, 0, edge_index[0])
        x_j = torch.index_select(data, 0, edge_index[1])
        x = torch.concat([x_i, x_j], dim=1)
        x = self.gcn_lin1(x)

        x_i = torch.index_select(node_embed, 0, edge_index[0])
        x_j = torch.index_select(node_embed, 0, edge_index[1])
        x1 = torch.concat([x_i, x_j], dim=1)
        x1 = self.gcn_lin2(x1)

        time_embed = self.time_embed(cur_t.long())
        week_embed = self.week_embed(cur_w.long())

        xf = torch.cat([embed, self.attr_lin1(attr), x, x1, time_embed, week_embed], dim=1)

        for lin in self.lins[:-1]:
            xf = lin(xf)
            xf = self.gelu(xf)

        xf = self.lins[-1](xf)

        return xf, x_rec

In [52]:
dataset = T4c22GeometricDataset(root=BASEDIR, city=CITY,
    edge_attributes=["speed_kph", "parsed_maxspeed", "length_meters", "counter_distance",
        'num_lanes', 'mean_pop', 'catering', 'leisure', 'shopping', 'tourism', 'traffic', 'transport',
        'pois_total', 'greeneries', 'water_bodies', 'built_up', 'other', "importance", "highway", "oneway",],
    split="train", fill=-1, normalize="zs", cachedir=Path(f"{BASEDIR}/cache"), idx=0)

In [53]:
dataset.__dict__.keys()

dict_keys(['root', 'transform', 'pre_transform', 'pre_filter', 'log', '_indices', 'cachedir', 'split', 'fill', 'normalize', 'city', 'limit', 'day_t_filter', 'torch_road_graph_mapping', 'day_t', 'cluster_map', 'edge_index', 'num_importance', 'num_highway', 'num_oneway', 'edge_attr', 'min_volume', 'max_volume', 'mean_volume', 'std'])

In [54]:
dataset[0]

Data(x=[59110, 4], y=[132414], volume_class=[132414], median_speed=[132414], max_speed=[132414], t=24, cluster=15, week=3)

In [56]:
len(dataset)

7040

In [57]:
dataset[-1]

Data(x=[59110, 4], y=[132414], volume_class=[132414], median_speed=[132414], max_speed=[132414], t=87, cluster=15, week=2)

In [58]:
dataset.edge_attr.shape

torch.Size([132414, 37])

In [59]:
test_dataset = T4c22GeometricDataset(root=BASEDIR, city=CITY,
                                     edge_attributes=["speed_kph", "parsed_maxspeed", "length_meters",
                                                      "counter_distance", "importance", "highway", "oneway", ],
                                     split="test", fill=-1,
                                     normalize="sz", cachedir=Path(f"{BASEDIR}/cache"), idx=0)

In [60]:
print("Dataset Size\t", len(dataset))
print("Test Dataset Size\t", len(test_dataset))
print("The statistics of training set are: Min [%d]\tMax [%d]\tMean [%.4f]\tStd[%.4f]" % (
    dataset.min_volume, dataset.max_volume, dataset.mean_volume, dataset.std))

Dataset Size	 7040
Test Dataset Size	 100
The statistics of training set are: Min [0]	Max [7262]	Mean [348.2630]	Std[289.1282]


In [61]:
spl = int(((0.8 * len(dataset)) // 2) * 2)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [spl, len(dataset) - spl])
print("Train Dataset Size\t", len(train_dataset))
print("Validation Dataset Size\t", len(val_dataset))

Train Dataset Size	 5632
Validation Dataset Size	 1408


In [62]:
train_dataset.dataset

T4c22GeometricDataset(7040)

In [63]:
next(iter(train_dataset))

Data(x=[59110, 4], y=[132414], volume_class=[132414], median_speed=[132414], max_speed=[132414], t=82, cluster=14, week=4)

In [64]:
next(iter(train_dataset))

Data(x=[59110, 4], y=[132414], volume_class=[132414], median_speed=[132414], max_speed=[132414], t=82, cluster=14, week=4)

In [65]:
city_class_fractions = class_fractions[CITY]
city_class_weights = torch.tensor(
    get_weights_from_class_fractions(
        [city_class_fractions['green'], city_class_fractions['yellow'],
         city_class_fractions['red']])).float()
print("City Class Weight\t", city_class_weights)

City Class Weight	 tensor([0.6210, 0.9486, 2.9807])


In [66]:
city_class_weights = city_class_weights.to(device)
edge_index = dataset.edge_index.to(device)
edge_attr = dataset.edge_attr.to(device)

In [67]:
num_edges = edge_index.shape[1]
num_attrs = edge_attr.shape[1]
num_nodes = np.max(edge_index.cpu().numpy()) + 1
print('num_nodes', num_nodes, "num_edges", num_edges, "num_attrs", num_attrs)

num_nodes 59110 num_edges 132414 num_attrs 37


In [68]:
index = torch.arange(0, num_edges).to(device)

In [69]:
model = RecLinear(num_edges, num_nodes, num_attrs, NUM_FEATURES, HIDDEN_CHANNELS,
                  NUM_EDGE_CLASSES, NUM_LAYERS, DROPOUT).to(device)

In [70]:
kfold = KFold(n_splits=5, shuffle=True)

In [71]:
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    print(fold, len(train_ids), len(test_ids))
    print(train_ids[:5])

0 5632 1408
[0 1 2 3 5]
1 5632 1408
[0 3 4 5 6]
2 5632 1408
[0 1 2 3 4]
3 5632 1408
[0 1 2 3 4]
4 5632 1408
[1 2 4 6 7]


In [72]:
train_ids, test_ids = next(iter(kfold.split(dataset)))

In [73]:
len(dataset)

7040

In [74]:
len(train_ids), len(test_ids)

(5632, 1408)

In [75]:
model.reset_parameters()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-3)
loss_f = torch.nn.CrossEntropyLoss(weight=city_class_weights, ignore_index=-1)
loss_mse = torch.nn.MSELoss()

min_loss = 10000

# print("fold", fold)
train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

In [76]:
train_subsampler

In [77]:
model.train()

RecLinear(
  (embed): Embedding(132414, 32)
  (node_embed): Embedding(59110, 32)
  (node_embed1): Embedding(59110, 4)
  (time_embed): Embedding(96, 32)
  (week_embed): Embedding(7, 32)
  (node_lin): Linear(in_features=4, out_features=32, bias=True)
  (node_lin1): Linear(in_features=64, out_features=32, bias=True)
  (attr_lin): Linear(in_features=37, out_features=32, bias=True)
  (attr_lin1): Sequential(
    (0): Linear(in_features=37, out_features=32, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=32, out_features=32, bias=True)
  )
  (lins): ModuleList(
    (0): Linear(in_features=192, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=3, bias=True)
  )
  (fc1): Linear(in_features=59110, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=32, bias=True)
  (fc3): Linear(in_features=256, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_f

In [78]:
losses = []
optimizer.zero_grad()

In [79]:
pbar = tqdm.tqdm(
    torch_geometric.loader.dataloader.DataLoader(dataset, batch_size=BATCH_SIZE,
                                                 num_workers=2, sampler=train_subsampler),
    "train",
    total=len(train_dataset) // BATCH_SIZE, )

train:   0%|          | 0/2816 [00:00<?, ?it/s]

In [80]:
count = 0
for data in pbar:
    data = data.to(device)
    data.x[data.x > 23.91] = 23.91
    data.x[data.x == -1] = nan_to_num_map[CITY]
    loss = 0.
    print(data)
    break

train:   0%|          | 0/2816 [00:01<?, ?it/s]

DataBatch(x=[118220, 4], y=[264828], volume_class=[264828], median_speed=[264828], max_speed=[264828], t=[2], cluster=[2], week=[2], batch=[118220], ptr=[3])


In [81]:
data

DataBatch(x=[118220, 4], y=[264828], volume_class=[264828], median_speed=[264828], max_speed=[264828], t=[2], cluster=[2], week=[2], batch=[118220], ptr=[3])

In [82]:
if (count == 0):
    lens = data.x.shape[0] // BATCH_SIZE
    lens1 = data.y.shape[0] // BATCH_SIZE
    count += 1

In [83]:
lens, lens1

(59110, 132414)

In [84]:
data.y.shape[0] // lens1

2

In [85]:
edge_index[0]

tensor([ 3751,  3751,  3751,  ..., 59107, 59108, 59109], device='cuda:0')

In [86]:
data.t[0]

tensor(55, device='cuda:0')

In [87]:
data.t[1]

tensor(54, device='cuda:0')

In [88]:
data.week[0], data.week[1]

(tensor(0, device='cuda:0'), tensor(0, device='cuda:0'))

In [89]:
i = 0
data.y[i * lens1:(i + 1) * lens1]

tensor([nan, 1., 1.,  ..., 1., nan, nan], device='cuda:0')

In [90]:
data.x[i * lens:(i + 1) * lens]

tensor([[ 4.0721e-01,  2.1699e-01,  6.0077e-03, -1.4744e-02],
        [ 1.0977e-01,  1.6165e-01,  1.3744e-01, -9.0963e-04],
        [-3.9520e-01, -3.0873e-01, -3.6061e-01, -3.9174e-01],
        ...,
        [-1.2100e+00, -1.2100e+00, -1.2100e+00, -1.2100e+00],
        [-1.2100e+00, -1.2100e+00, -1.2100e+00, -1.2100e+00],
        [-1.2100e+00, -1.2100e+00, -1.2100e+00, -1.2100e+00]], device='cuda:0')

In [91]:
for i in range(data.y.shape[0] // lens1):
    t = data.t[i]
    cur_t = torch.ones_like(edge_index[0]) * t
    week = data.week[i]
    cur_week = torch.ones_like(edge_index[0]) * week

    y = data.y[i * lens1:(i + 1) * lens1].nan_to_num(-1)
    x = data.x[i * lens:(i + 1) * lens]
    y_hat, x_rec = model(index, edge_index, x,
                         edge_attr, cur_t, cur_week)
    y = y.long()

    train_index = torch.nonzero(torch.sum(x, dim=1) != nan_to_num_map[CITY] * 4).squeeze()

    rec_loss = loss_mse(x[train_index], x_rec[train_index])
    acc_loss = loss_f(y_hat, y)

    loss += rec_loss + acc_loss

In [92]:
edge_index

tensor([[ 3751,  3751,  3751,  ..., 59107, 59108, 59109],
        [16812, 16813, 58530,  ..., 19129, 58304, 21202]], device='cuda:0')

In [93]:
x

tensor([[ 0.0959,  0.2827,  0.2343,  0.1305],
        [ 0.1236,  0.2032,  0.2447,  0.2204],
        [-0.4021, -0.3572, -0.3087, -0.3053],
        ...,
        [-1.2100, -1.2100, -1.2100, -1.2100],
        [-1.2100, -1.2100, -1.2100, -1.2100],
        [-1.2100, -1.2100, -1.2100, -1.2100]], device='cuda:0')

In [94]:
edge_attr

tensor([[-0.4703, -0.4703, -0.9918,  ...,  0.0000,  1.0000,  0.0000],
        [-0.4703, -0.4703, -0.9728,  ...,  0.0000,  1.0000,  0.0000],
        [-0.4703, -0.4703, -0.9649,  ...,  0.0000,  0.0000,  1.0000],
        ...,
        [-0.4703, -0.4703, -0.9980,  ...,  0.0000,  0.0000,  1.0000],
        [-0.6932, -0.6932, -0.9966,  ...,  0.0000,  0.0000,  1.0000],
        [-0.4703, -0.4703, -0.9969,  ...,  0.0000,  0.0000,  1.0000]],
       device='cuda:0')

In [95]:
edge_attr.shape

torch.Size([132414, 37])

In [44]:
cur_t

tensor([44, 44, 44,  ..., 44, 44, 44], device='cuda:0')

In [45]:
cur_week

tensor([4, 4, 4,  ..., 4, 4, 4], device='cuda:0')

In [38]:
week

tensor(1, device='cuda:0')

In [59]:
mask_idx = (torch.sum(x, dim=1, keepdim=True) != nan_to_num_map[CITY] * 4).type(torch.float)

In [62]:
torch.sum(x, dim=1, keepdim=True).shape

torch.Size([59110, 1])

In [63]:
xmax = 23.91
xmin = nan_to_num_map[CITY]
x_norm = (x - xmin) / (xmax - xmin)

In [64]:
x_norm

tensor([[0.0316, 0.0378, 0.0411, 0.0389],
        [0.0506, 0.0503, 0.0498, 0.0553],
        [0.0439, 0.0400, 0.0436, 0.0423],
        ...,
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000]], device='cuda:0')

In [69]:
ratio = 0.8 + 0.4 * np.random.rand(1)[0]
ratio

1.1627586468387787

In [70]:
x_norm = x_norm * ratio

In [73]:
torch.rand_like(x_norm[:, 0:1])

tensor([[0.2663],
        [0.5309],
        [0.0177],
        ...,
        [0.0608],
        [0.5850],
        [0.8980]], device='cuda:0')

In [77]:
drop_idx = (torch.rand_like(x_norm[:, 0:1]) > 0.4).type(torch.float)
drop_idx

tensor([[0.],
        [1.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]], device='cuda:0')

In [78]:
x_norm = x_norm * drop_idx

In [79]:
x_norm.shape

torch.Size([59110, 4])

In [80]:
x_norm = torch.transpose(x_norm, 0, 1)
x_norm.shape

torch.Size([4, 59110])

In [85]:
mu, log_var = model.encode(x_norm)

In [87]:
z = model.reparameterize(mu, log_var)

In [88]:
x_rec = model.decode(z)

In [89]:
x_rec

tensor([[ 0.0526,  0.0582,  0.0420,  ...,  0.1324, -1.0387, -0.1499],
        [ 0.0536,  0.0569,  0.0414,  ...,  0.1187, -1.0274, -0.1508],
        [ 0.0539,  0.0579,  0.0422,  ...,  0.1143, -1.0406, -0.1488],
        [ 0.0537,  0.0576,  0.0421,  ...,  0.1118, -1.0372, -0.1484]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [90]:
x_rec = x_rec / ratio

In [93]:
x_rec = torch.transpose(x_rec, 0, 1)
x_rec.shape

torch.Size([59110, 4])

In [94]:
x_rec = x_rec * (xmax - xmin) + xmin

In [95]:
x_rec

tensor([[-7.3412e-02, -5.2531e-02, -4.5680e-02, -5.0454e-02],
        [ 4.7557e-02,  1.9235e-02,  4.1158e-02,  3.4510e-02],
        [-3.0330e-01, -3.1479e-01, -2.9906e-01, -3.0113e-01],
        ...,
        [ 1.6503e+00,  1.3539e+00,  1.2588e+00,  1.2051e+00],
        [-2.3650e+01, -2.3406e+01, -2.3691e+01, -2.3618e+01],
        [-4.4491e+00, -4.4678e+00, -4.4247e+00, -4.4170e+00]], device='cuda:0',
       grad_fn=<AddBackward0>)

In [96]:
x_rec1 = mask_idx * x + (1 - mask_idx) * x_rec

In [97]:
x_rec1

tensor([[ -0.4160,  -0.2603,  -0.1773,  -0.2326],
        [  0.0613,   0.0544,   0.0406,   0.1789],
        [ -0.1081,  -0.2050,  -0.1150,  -0.1462],
        ...,
        [  1.6503,   1.3539,   1.2588,   1.2051],
        [-23.6502, -23.4062, -23.6905, -23.6179],
        [ -4.4491,  -4.4678,  -4.4247,  -4.4170]], device='cuda:0',
       grad_fn=<AddBackward0>)

In [39]:
y

tensor([-1, -1, -1,  ..., -1, -1, -1], device='cuda:0')

In [42]:
y.unique()

tensor([-1,  0,  1,  2], device='cuda:0')

In [43]:
x

tensor([[-0.4160, -0.2603, -0.1773, -0.2326],
        [ 0.0613,  0.0544,  0.0406,  0.1789],
        [-0.1081, -0.2050, -0.1150, -0.1462],
        ...,
        [-1.2100, -1.2100, -1.2100, -1.2100],
        [-1.2100, -1.2100, -1.2100, -1.2100],
        [-1.2100, -1.2100, -1.2100, -1.2100]], device='cuda:0')

# Testing inference mode

In [29]:
model.eval()

RecLinear(
  (embed): Embedding(132414, 32)
  (node_embed): Embedding(59110, 32)
  (node_embed1): Embedding(59110, 4)
  (time_embed): Embedding(96, 32)
  (week_embed): Embedding(7, 32)
  (node_lin): Linear(in_features=4, out_features=32, bias=True)
  (node_lin1): Linear(in_features=64, out_features=32, bias=True)
  (attr_lin): Linear(in_features=39, out_features=32, bias=True)
  (attr_lin1): Sequential(
    (0): Linear(in_features=39, out_features=32, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=32, out_features=32, bias=True)
  )
  (lins): ModuleList(
    (0): Linear(in_features=192, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=3, bias=True)
  )
  (fc1): Linear(in_features=59110, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=32, bias=True)
  (fc3): Linear(in_features=256, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_f

In [30]:
idx = 0
data = next(iter(test_dataset))
data

Data(x=[59110, 4], t=44, cluster=0, week=4)

In [31]:
data = data.to(device)

In [32]:
data.x

tensor([[290., 284., 313., 311.],
        [415., 454., 440., 442.],
        [282., 301., 291., 296.],
        ...,
        [ -1.,  -1.,  -1.,  -1.],
        [ -1.,  -1.,  -1.,  -1.],
        [ -1.,  -1.,  -1.,  -1.]], device='cuda:0')

In [33]:
data.x[data.x > 23.91] = 23.91
data.x

tensor([[23.9100, 23.9100, 23.9100, 23.9100],
        [23.9100, 23.9100, 23.9100, 23.9100],
        [23.9100, 23.9100, 23.9100, 23.9100],
        ...,
        [-1.0000, -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000, -1.0000]], device='cuda:0')

In [34]:
data.x[data.x == -1] = nan_to_num_map[CITY]
data.x

tensor([[23.9100, 23.9100, 23.9100, 23.9100],
        [23.9100, 23.9100, 23.9100, 23.9100],
        [23.9100, 23.9100, 23.9100, 23.9100],
        ...,
        [-1.2100, -1.2100, -1.2100, -1.2100],
        [-1.2100, -1.2100, -1.2100, -1.2100],
        [-1.2100, -1.2100, -1.2100, -1.2100]], device='cuda:0')

In [35]:
t = data.t
t

44

In [36]:
cur_t = torch.ones_like(edge_index[0]) * t
cur_t

tensor([44, 44, 44,  ..., 44, 44, 44], device='cuda:0')

In [37]:
week = data.week
week

4

In [38]:
cur_week = torch.ones_like(edge_index[0]) * week

In [43]:
save_path = "save/cc/class_L3_H32_F-1_Nzs_B2_e20/london_2/"
y_pred = 0.
for fold in range(5):
    for mm in range(16, 21):
        print(f"{save_path}model_{fold}_{mm:03d}.pt")
        model.load_state_dict(torch.load(f"{save_path}model_{fold}_{mm:03d}.pt", map_location='cuda:0'))
        model.eval()
        for ii in range(5):
            y_hat, _ = model(index, edge_index, data.x, edge_attr, cur_t, cur_week)
            y_hat = y_hat.detach()
            print(y_hat.shape, y_hat[:5])
            y_pred += y_hat
        break
y_pred /= 125.

save/cc/class_L3_H32_F-1_Nzs_B2_e20/london_2/model_0_016.pt
torch.Size([132414, 3]) tensor([[ 3.0692,  1.4098, -3.5911],
        [ 3.8815,  2.5751, -5.1036],
        [ 5.3915,  2.3949, -6.0946],
        [ 3.8038,  0.7124, -3.5893],
        [ 4.3619,  1.6560, -4.7951]], device='cuda:0')
torch.Size([132414, 3]) tensor([[ 2.6913,  1.1739, -3.0897],
        [ 3.6341,  2.7222, -4.9575],
        [ 5.9121,  2.6427, -6.6754],
        [ 4.6326,  0.8336, -4.2185],
        [ 4.5391,  1.8040, -5.0426]], device='cuda:0')
torch.Size([132414, 3]) tensor([[ 2.2826,  0.2595, -1.8338],
        [ 4.0678,  2.4912, -5.0494],
        [ 5.7483,  2.1528, -6.1262],
        [ 5.5406,  1.3419, -5.2464],
        [ 5.3434,  1.7030, -5.5393]], device='cuda:0')
torch.Size([132414, 3]) tensor([[ 3.2593,  1.7216, -4.0339],
        [ 4.3313,  3.6788, -6.2417],
        [ 6.7294,  3.5289, -8.0126],
        [ 3.8311,  0.1867, -3.1252],
        [ 3.6785,  1.7776, -4.3572]], device='cuda:0')
torch.Size([132414, 3]) tensor([

In [28]:
y_pred

tensor([[  6.2839,  -0.3915,  -8.4779],
        [  5.2118,   1.4126,  -7.4263],
        [  5.9455,   1.2316,  -7.8422],
        ...,
        [ -2.2475,   1.5776,   2.1567],
        [ 13.5077,   7.2808, -19.1663],
        [  5.2458,   1.9448,  -7.3630]], device='cuda:0')

In [29]:
df = test_dataset.torch_road_graph_mapping._torch_to_df_cc(data=y_pred, day="test", t=idx)

In [31]:
df.head()

,logit_green,logit_yellow,logit_red,u,v,day,t
0,6.283916,-0.391485,-8.477943,78112,25508583,test,0
1,5.211779,1.412575,-7.426338,78112,25508584,test,0
2,5.945462,1.231592,-7.842179,78112,3257621681005534125,test,0
3,1.780440,0.607034,-2.405866,99936,2146383887,test,0
4,1.419122,0.663258,-1.920764,99936,4544836433,test,0


In [ ]:
dfs = []
for idx, data in tqdm.tqdm(enumerate(test_dataset), total=len(test_dataset)):
    data = data.to(device)
    data.x[data.x > 23.91] = 23.91
    data.x[data.x == -1] = nan_to_num_map[opt['city']]

    t = data.t
    cur_t = torch.ones_like(edge_index[0]) * t
    week = data.week
    cur_week = torch.ones_like(edge_index[0]) * week

    if (opt['city'] == 'melbourne'):
        y_pred = 0.
        for fold in range(5):
            model.load_state_dict(torch.load(f"{opt['save_path']}model_best_{fold}.pt", map_location='cuda:0'))
            model.eval()
            for ii in range(5):
                y_hat, _ = model(index, edge_index, data.x, edge_attr, cur_t, cur_week)
                y_hat = y_hat.detach()
                y_pred += y_hat
        y_pred /= 25.
    else:
        y_pred = 0.
        for fold in range(5):
            for mm in range(16, 21):
                model.load_state_dict(torch.load(f"{opt['save_path']}model_{fold}_{mm:03d}.pt", map_location='cuda:0'))
                model.eval()
                for ii in range(5):
                    y_hat, _ = model(index, edge_index, data.x, edge_attr, cur_t, cur_week)
                    y_hat = y_hat.detach()
                    y_pred += y_hat
        y_pred /= 125.

    df = test_dataset.torch_road_graph_mapping._torch_to_df_cc(data=y_pred, day="test", t=idx)
    dfs.append(df)
df = pd.concat(dfs)
df["test_idx"] = df["t"]
del df["day"]
del df["t"]

submission = df
# print(submission.head(20))

# (BASEDIR / "submissions" / opt['submission_name'] / opt['city'] / "labels").mkdir(exist_ok=True, parents=True)
# table = pa.Table.from_pandas(submission)
# pq.write_table(table, BASEDIR / "submissions" / opt['submission_name'] / opt[
#     'city'] / "labels" / f"cc_labels_test.parquet", compression="snappy")


In [33]:
df

,logit_green,logit_yellow,logit_red,u,v,day,t
0,6.283916,-0.391485,-8.477943,78112,25508583,test,0
1,5.211779,1.412575,-7.426338,78112,25508584,test,0
2,5.945462,1.231592,-7.842179,78112,3257621681005534125,test,0
3,1.780440,0.607034,-2.405866,99936,2146383887,test,0
4,1.419122,0.663258,-1.920764,99936,4544836433,test,0
...,...,...,...,...,...,...,...
132409,-0.218619,1.644511,-0.039306,4595139612105786518,8842311879,test,0
132410,2.548334,0.911821,-2.964944,8230831116681660864,1149426165,test,0
132411,-2.247471,1.577633,2.156674,1688447984145568529,26559620,test,0
132412,13.507724,7.280828,-19.166265,3771856370570656347,9402106041,test,0
